<a href="https://colab.research.google.com/github/Jashwanth63/Custom-Model-for-classification-of-Image-Data-without-using-Convolutional-Neural-Networks/blob/main/Deep_Learning_Assignment_2022_Jashwanth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
from torchvision.datasets import FashionMNIST

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [89]:
import torchvision
from torchvision import transforms

In [91]:
def download_data(b_size=64):  
    """Download the Fashion-MNIST dataset and then load it into memory."""
    mnist_train = torchvision.datasets.FashionMNIST(".", train = True, transform = transforms.ToTensor(), target_transform= transforms.Lambda(lambda x:torch.zeros(
            10, dtype = torch.float64).scatter_(dim = 0, index= torch.tensor(x), value = 1)) ,download = True)
    mnist_train = torch.utils.data.DataLoader(mnist_train, batch_size = b_size, shuffle = True, num_workers = 3)

    mnist_test = torchvision.datasets.FashionMNIST(".", train = False, transform = transforms.ToTensor(), target_transform = transforms.Lambda(
        lambda x:torch.zeros(10, dtype = torch.float64).scatter_(dim=0, index = torch.tensor(x), value = 1)), download = True)
    mnist_test = torch.utils.data.DataLoader(mnist_test, batch_size=b_size, shuffle = True, num_workers = 3)
    return mnist_train, mnist_test  

In [92]:
training,  testing = download_data(b_size=128)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for i, batch in enumerate(training):
  if i==1:
    break
  for image in batch:
    reshaped = image.view(28,28)
    reshaped[0:3, ]

In [93]:
#Patch Matrix Dimensions = Image_Dimension / Patch_dimension. Example: Patch => (2,2), Image => (10,10)
#Total_Num_of_patches = 5*5 = 25, Each Patch will have 4 elements, if patch size is 2X2, so Total Elements for one Image = 25*4 = 100

#Similarly, for an image of size 28X28, If patch_size is 4X4, then total Number of patches = 7*7 = 28.
#Each patch has a dimension of 4, So 8 elements, 28*8 = 224 Elements per Image. 

def patch_maker(data, patch_size = 4):
  for i, batch in enumerate(data):
    for image in batch:
      i


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


468


In [96]:
a = np.array([[1,2,3,4,5,6],[7,8,9,10,11,12],[13,14,15,16,17,18]])

In [99]:
b = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

In [100]:
b

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [102]:
b.shape[0]

4

In [98]:
j = 2
for i in range(b.shape[0] / j)**2:
  print(a[0:j, 0:j])
  j +=2


array([[1, 2],
       [7, 8]])